In [1]:
import json
import os
import csv
import pandas as pd
import numpy as np

In [28]:
## variables for dota2
game = 'dota2'
redone_filter = ['Kill', 'Team Fight', 'Push', 'Gank', 'Farm', 'Group Fight', 'Roshan', 'Roam', 'Split Push']
titles = ['Kill', 'Team Fight', 'Push', 'Gank', 'Farm', 'Group Fight', 'Roshan', 'Roam', 'Split Push', 'No']
heading = ['Push', 'Split Push', 'Roshan', 'No Label', 'TimeStamp', 'Push', 'Split Push', 'Roshan', 'No Label', 'TimeStamp']
rename_title = {}


# game = 'boomtown'
# redone_filter = ['Explosive Inner', 'Explosive Quick Mine', 'Explosive Tunneling', 'Exploring', 'Tunneling Pickaxe', 'Pickaxing for Gold', 'Explosive Miss']
# titles = ['Explosive Quick Mine', 'Explosive Inner', 'Explosive Tunneling', 'Exploring', 'Tunneling Pickaxe', 'Pickaxing for Gold', 'Explosive Miss', 'No']
# # heading = ['timepoint', 'Explosive Inner', 'Explosive Quick Mine', 'Explosive Tunneling', 'Exploring', 'Tunneling Pickaxe', 'Pickaxing for Gold', 'Explosive Miss', 'No/Other Label', 'Explosive Inner', 'Explosive Quick Mine', 'Explosive Tunneling', 'Exploring', 'Tunneling Pickaxe', 'Pickaxing for Gold', 'Explosive Miss', 'No/Other Label']
# # heading = ['Jimmy Explosive Quick Mine', 'Jimmy Explosive Inner', 'Jimmy Explosive Tunneling', 'Jimmy Exploring', 'Jimmy Tunneling Pickaxe', 'Jimmy Pickaxing for Gold', 'Jimmy Explosive Miss', 'Jimmy No label', 'Jimmy TimeStamp', 'Erica Explosive Quick Mine', 'Erica Explosive Inner', 'Erica Explosive Tunneling', 'Erica Exploring', 'Erica Tunneling Pickaxe', 'Erica Pickaxing for Gold', 'Erica Explosive Miss', 'Erica No label', 'Erica TimeStamp']
# heading = ['Explosive Quick Mine', 'Explosive Inner', 'Explosive Tunneling', 'Exploring', 'Tunneling Pickaxe', 'Pickaxing for Gold', 'Explosive Miss', 'No label', 'TimeStamp', 'Explosive Quick Mine', 'Explosive Inner', 'Explosive Tunneling', 'Exploring', 'Tunneling Pickaxe', 'Pickaxing for Gold', 'Explosive Miss', 'No label', 'TimeStamp']
# rename_title = {'Explosive - Inner': 'Explosive Inner', 'Tunneling - Explosive' :'Explosive Tunneling'}

original_label_dir = './data/'+game+'/original/'
redone_label_dir = './data/'+game+'/redone/'

orginal_label_initial = 'Erica'  # Original
redone_label_initial = 'Jimmy'  # Redone

irr_table_output_file = './data/'+game+'/'+game+'_irr_table.csv'
irr_matrix_output_file = './data/'+game+'/'+game+'_irr_matrix.csv'

same_labels = []
different_labels = []

In [29]:
def get_name_and_extension(file_name):
    """
    get name and extension of a file_name
    :param file_name:
    :return:
    """
    tokens = file_name.split('.')
    return {'name': '.'.join(tokens[0:-1]), 'ext': tokens[-1]}

In [30]:
def check_overlap(t1, t2):
    s1 = t1[0]
    f1 = t1[1]
    s2 = t2[0]
    f2 = t2[1]
#     print(s1, f1, s2, f2)
    
    if s2<=s1 and f1<=f2:
#         print('c1')
        return f1-s1+1
    elif s1<=s2 and f2<=f1:
#         print('c2')
        return f2-s2+1
    elif s2<=s1 and f2<=f1 and s1<=f2:
#         print('c3')
        return f2-s1+1
    elif s1<=s2 and f1<=f2 and s2<=f1:
#         print('c4')
        return f1-s2+1
    
    return -1

In [31]:
# def analyze_labels(original_labels, redone_labels):
#     redone_labels_filtered = []
#     for label in redone_labels:
#         if label['title'] in redone_filter:
#             redone_labels_filtered.append(label)

#     for label in redone_labels_filtered:
#         print(label['title'], label['timestamp_range'], label['author'], label['units'])
#         for ori_label in original_labels:
#             overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
#             if  overlap > -1:
#                 print('-->', ori_label['title'], ori_label['timestamp_range'], ori_label['author'], ori_label['units'])
            
#         break


In [32]:
def analyze_single_player(original_labels, redone_labels, table_info):
    
    # only looking at same titles
    for label in redone_labels:
#         print(label['title'], label['timestamp_range'], label['author'], label['units'])
        label['longest'] = -1
        for ori_label in original_labels:
            overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
            
            if  overlap > -1:
#                 print('\t', ori_label['title'], ori_label['timestamp_range'], ori_label['author'], ori_label['units'], overlap)
                temp_title = ori_label['title']
                if ori_label['title'] in rename_title:    
                    temp_title = rename_title[ori_label['title']]
                
                if temp_title == label['title'] and overlap>t_hold_same:
#                     print('----FOUND----')
                    ori_label['checked'] = True
                    label['checked'] = True
                    table_info.append((label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range']))
                    print(0, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']')
                    if label['longest'] < overlap:
                        label['longest'] = overlap
    
    # looking at different titles
    for label in redone_labels:
#         print(label['title'], label['timestamp_range'], label['author'], label['units'])
        if 'checked' in label:
            for ori_label in original_labels:
                if label['title'] != ori_label['title']:
                    overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
                    if overlap > label['longest'] and overlap > t_hold_diff:
                        temp_title = ori_label['title']
                        if ori_label['title'] in rename_title:    
                            temp_title = rename_title[ori_label['title']]
                        ori_label['checked'] = True
                        table_info.append((label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range']))
                        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                        print('S/D', overlap, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']')
                        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
            continue
        
        i = 0
        longest = -1
        longest_index = -1
        for ori_label in original_labels:
            overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
            
            if  overlap > -1:
#                 print('\t', ori_label['title'], ori_label['timestamp_range'], ori_label['author'], ori_label['units'], overlap)
                temp_title = ori_label['title']
                if ori_label['title'] in rename_title:    
                    temp_title = rename_title[ori_label['title']]

                if 'checked' not in ori_label and temp_title in redone_filter and overlap>t_hold_diff:
                    if overlap > longest:
                        longest = overlap
                        longest_index = i

            i += 1
        
        if longest_index > -1:
            original_labels[longest_index]['checked'] = True
            label['checked'] = True
            table_info.append((label['title'], original_labels[longest_index]['title'], label['timestamp_range'], original_labels[longest_index]['timestamp_range']))
            print(1, '[', label['title'], original_labels[longest_index]['title'], label['timestamp_range'], original_labels[longest_index]['timestamp_range'], ']')

        
    # generate No labels for unchecked redone_labels
    for label in redone_labels:
        if 'checked' not in label:
            table_info.append((label['title'], 'No', label['timestamp_range'], -1))
            print(2, '[', label['title'], 'No', label['timestamp_range'], -1, ']')

    # generate No labels for unchecked original_labels
    for ori_label in original_labels:
        temp_title = ori_label['title']
        if ori_label['title'] in rename_title:    
            temp_title = rename_title[ori_label['title']]
            
        if temp_title in redone_filter:
#             if 'checked' in ori_label:
#                 print(ori_label)
            if 'checked' not in ori_label:
                table_info.append(('No', temp_title, -1, ori_label['timestamp_range']))
                print(3, '[', 'No', temp_title, -1, ori_label['timestamp_range'], ']')
                

In [51]:
###### label overlapping analyze
def analyze_overlaps_org(original_labels, redone_labels):
    for label in redone_labels:
#         print(label['title'], label['timestamp_range'], label['author'], label['units'])
        for ori_label in original_labels:
            overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
            
            if  overlap > -1:
#                 print('\t', ori_label['title'], ori_label['timestamp_range'], ori_label['author'], ori_label['units'], overlap)
                temp_title = ori_label['title']
                if ori_label['title'] in rename_title:    
                    temp_title = rename_title[ori_label['title']]
                
                if temp_title == label['title']:
                    same_labels.append(overlap)
                    print(overlap, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']')
                    if overlap<4:
                        print('-------')
                else:
                    different_labels.append(overlap)

In [34]:
###### label overlapping analyze
def analyze_overlaps(original_labels, redone_labels):
    # only looking at same titles
    for label in redone_labels:
#         print(label['title'], label['timestamp_range'], label['author'], label['units'])
        for ori_label in original_labels:
            overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
            
            if  overlap > -1:
#                 print('\t', ori_label['title'], ori_label['timestamp_range'], ori_label['author'], ori_label['units'], overlap)
                temp_title = ori_label['title']
                if ori_label['title'] in rename_title:    
                    temp_title = rename_title[ori_label['title']]
                
                if temp_title == label['title']:# and overlap>2:
                    same_labels.append(overlap)
                    ori_label['checked'] = True
                    label['checked'] = True
                    print(overlap, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']')
                    if overlap<4:
                        print('-------')

    print('****************************')
    # looking at different titles
    for label in redone_labels:
#         print(label['title'], label['timestamp_range'], label['author'], label['units'])
        if 'checked' in label:
            continue
        
        for ori_label in original_labels:
            overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
            
            if  overlap > -1:
#                 print('\t', ori_label['title'], ori_label['timestamp_range'], ori_label['author'], ori_label['units'], overlap)
                temp_title = ori_label['title']
                if ori_label['title'] in rename_title:    
                    temp_title = rename_title[ori_label['title']]
                

                if 'checked' not in ori_label and temp_title in redone_filter:
                    different_labels.append(overlap)
                    ori_label['checked'] = True
                    label['checked'] = True
                    print(overlap, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']')
        

In [35]:
###### label overlapping analyze for same and different label overlap times
def analyze_overlap_values(original_labels, redone_labels):
    for label in redone_labels:
        sm = None
        df = None
        smv = -1
        dfv = -1
        for ori_label in original_labels:
            overlap = check_overlap(ori_label['timestamp_range'], label['timestamp_range'])
            
            if  overlap > -1:
#                 print('\t', ori_label['title'], ori_label['timestamp_range'], ori_label['author'], ori_label['units'], overlap)
                temp_title = ori_label['title']
                if ori_label['title'] in rename_title:    
                    temp_title = rename_title[ori_label['title']]
                
                if temp_title == label['title']:
                    if smv < overlap:
                        smv = overlap
                        sm = ['Same', overlap, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']']
#                     print('Same', overlap, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']')
                else:
                    if dfv < overlap:
                        dfv = overlap
                        df = ['Diff', overlap, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']']
#                     print('Diff', overlap, '[', label['title'], temp_title, label['timestamp_range'], ori_label['timestamp_range'], ']')

        if sm is not None and df is not None and smv < dfv:
            print(sm)
            print(df)
            print('*****')
    
    for ol in original_labels:
        if 'testing' in ol:
            print('#############', ol['title'])

In [36]:
def separate_players(labels):
    player_labels = {}
    for label in labels:
        for unit in label['units']:
            if unit not in player_labels:
                player_labels[unit] = [label]
            else:
                player_labels[unit].append(label)
    return player_labels

In [52]:
def analyze_labels_by_players(original_labels, redone_labels, table_info):
    # filtering
    redone_labels_filtered = []
    
    for label in redone_labels:
        temp_title = label['title']
        if label['title'] in rename_title:  
            temp_title = rename_title[label['title']]
            
        if temp_title in redone_filter:
            redone_labels_filtered.append(label)
            
    original_player_labels = separate_players(original_labels)
    redone_player_labels = separate_players(redone_labels_filtered)
    
    for pl in redone_player_labels:
        if pl in original_player_labels:
            print('-----------------------------', pl, '-----------------------------')
#             analyze_single_player(original_player_labels[pl], redone_player_labels[pl], table_info)
#             analyze_overlaps(original_player_labels[pl], redone_player_labels[pl])
            analyze_overlaps_org(original_player_labels[pl], redone_player_labels[pl])
#             analyze_overlap_values(original_player_labels[pl], redone_player_labels[pl])
        
#     print('---')
#     for pl in original_player_labels:
#         print(pl)
    return table_info

In [53]:
def process_files(original_folder, redone_folder):
    table_info = []
    
    for subdir, dirs, files in os.walk(redone_folder):
        ind = 1
        for filename in files:
            name_extension = get_name_and_extension(filename)
            name = name_extension['name']
            ext = name_extension['ext'].lower()

            if ext == 'json':
                print(ind, ":", name)
                if os.path.isfile(original_folder+filename):
                    print('----> ', filename)
                    with open(redone_folder+filename, 'r') as redone_label_data:
                        redone_labels = json.load(redone_label_data)
                        with open(original_folder+filename, 'r') as original_label_data:
                            original_labels = json.load(original_label_data)
                            
                            table_info = analyze_labels_by_players(original_labels, redone_labels, table_info)
       
                else: 
                    continue
                    
            ind += 1

    return table_info
#     return same_labels, different_labels

In [54]:
same_labels = []
different_labels = []
table_info = process_files(original_label_dir, redone_label_dir)

2 : 2593668385
---->  2593668385.json
----------------------------- heroalchemist1 -----------------------------
97 [ Farm Farm [610, 706] [609, 707] ]
97 [ Farm Farm [726, 861] [730, 826] ]
12 [ Farm Farm [726, 861] [846, 857] ]
17 [ Farm Farm [896, 914] [898, 914] ]
14 [ Farm Farm [928, 947] [933, 946] ]
36 [ Farm Farm [999, 1177] [1002, 1037] ]
75 [ Farm Farm [999, 1177] [1047, 1121] ]
6 [ Kill Kill [1177, 1188] [1181, 1186] ]
79 [ Farm Farm [1204, 1384] [1204, 1282] ]
31 [ Farm Farm [1204, 1384] [1299, 1329] ]
43 [ Farm Farm [1204, 1384] [1342, 1384] ]
14 [ Gank Gank [1476, 1489] [1473, 1489] ]
9 [ Group Fight Group Fight [1824, 1838] [1830, 1838] ]
27 [ Farm Farm [1396, 1490] [1399, 1425] ]
15 [ Farm Farm [1396, 1490] [1431, 1445] ]
10 [ Farm Farm [1396, 1490] [1470, 1479] ]
14 [ Farm Farm [1535, 1809] [1537, 1550] ]
56 [ Farm Farm [1535, 1809] [1566, 1621] ]
31 [ Farm Farm [1535, 1809] [1636, 1666] ]
66 [ Farm Farm [1535, 1809] [1674, 1739] ]
61 [ Farm Farm [1535, 1809] [1749, 18

139 [ Roam Roam [2455, 2593] [2451, 2601] ]
75 [ Roam Roam [2669, 2743] [2626, 2778] ]
45 [ Roshan Roshan [2977, 3024] [2977, 3021] ]
28 [ Push Push [3206, 3235] [3207, 3234] ]
69 [ Roam Roam [2792, 2977] [2909, 3133] ]
110 [ Roam Roam [3024, 3206] [2909, 3133] ]
68 [ Roam Roam [3024, 3206] [3139, 3260] ]
26 [ Roam Roam [3235, 3260] [3139, 3260] ]
20 [ Team Fight Team Fight [3323, 3352] [3333, 3352] ]
30 [ Roshan Roshan [3610, 3646] [3615, 3644] ]
52 [ Roam Roam [3272, 3323] [3261, 3338] ]
212 [ Roam Roam [3352, 3610] [3359, 3570] ]
27 [ Roam Roam [3352, 3610] [3584, 3662] ]
17 [ Roam Roam [3646, 3729] [3584, 3662] ]
62 [ Roam Roam [3646, 3729] [3668, 3729] ]
5 [ Kill Kill [4147, 4161] [4156, 4160] ]
186 [ Roam Roam [3730, 3920] [3730, 3915] ]
48 [ Roam Roam [3996, 4147] [4100, 4156] ]
43 [ Team Fight Team Fight [4199, 4258] [4199, 4241] ]
37 [ Roshan Roshan [4276, 4315] [4279, 4316] ]
21 [ Team Fight Team Fight [4342, 4365] [4344, 4364] ]
7 [ Team Fight Team Fight [4409, 4438] [4421, 

12 [ Gank Gank [1130, 1143] [1132, 1143] ]
93 [ Farm Farm [870, 1220] [870, 962] ]
35 [ Farm Farm [870, 1220] [978, 1012] ]
56 [ Farm Farm [870, 1220] [1020, 1075] ]
113 [ Farm Farm [870, 1220] [1095, 1207] ]
110 [ Farm Farm [1221, 1353] [1244, 1362] ]
16 [ Farm Farm [1401, 1478] [1416, 1431] ]
69 [ Farm Farm [1500, 1571] [1503, 1571] ]
62 [ Farm Farm [1664, 1773] [1667, 1728] ]
20 [ Farm Farm [1664, 1773] [1751, 1770] ]
47 [ Team Fight Team Fight [1964, 2010] [1959, 2010] ]
32 [ Team Fight Team Fight [2184, 2216] [2185, 2216] ]
32 [ Team Fight Team Fight [2227, 2261] [2228, 2259] ]
30 [ Farm Farm [1923, 1964] [1923, 1952] ]
15 [ Farm Farm [2010, 2184] [2020, 2034] ]
102 [ Farm Farm [2010, 2184] [2050, 2151] ]
13 [ Push Push [2328, 2370] [2357, 2369] ]
2 [ Push Push [2328, 2370] [2369, 2397] ]
-------
17 [ Push Push [2481, 2515] [2498, 2514] ]
46 [ Farm Farm [2422, 2481] [2436, 2487] ]
26 [ Farm Farm [2540, 2624] [2598, 2623] ]
12 [ Team Fight Team Fight [2808, 2819] [2808, 2826] ]
11 

In [55]:
##### analyze label overlaps
same_labels = np.array(same_labels)
different_labels = np.array(different_labels)
print('Same Labels: ', len(same_labels))
print('min', np.min(same_labels))
print('max', np.max(same_labels))
print('avg', np.average(same_labels))
print('std', np.std(same_labels))
print('mean', np.mean(same_labels))

print('\nDifferent Labels: ', len(different_labels))
print('min', np.min(different_labels))
print('max', np.max(different_labels))
print('avg', np.average(different_labels))
print('std', np.std(different_labels))
print('mean', np.mean(different_labels))

Same Labels:  889
min 1
max 240
avg 39.6749156355
std 37.8522830557
mean 39.6749156355

Different Labels:  1104
min 1
max 130
avg 12.1748188406
std 13.4855417735
mean 12.1748188406


In [56]:
# np.where((same_labels==1))
print('\tCounts')
print('Overlap','Same','\t','Diff')
for i in range(0,np.max(same_labels)+1):
    print(i, '\t', np.count_nonzero(same_labels==i), end =" ")
    if i <np.max(different_labels)+1:
        print('\t', np.count_nonzero(different_labels==i))
        continue
    print("")

	Counts
Overlap Same 	 Diff
0 	 0 	 0
1 	 8 	 159
2 	 1 	 98
3 	 1 	 70
4 	 3 	 68
5 	 13 	 57
6 	 13 	 41
7 	 14 	 33
8 	 14 	 66
9 	 31 	 13
10 	 22 	 45
11 	 25 	 32
12 	 27 	 28
13 	 11 	 13
14 	 28 	 42
15 	 31 	 22
16 	 29 	 29
17 	 9 	 46
18 	 13 	 20
19 	 20 	 16
20 	 36 	 7
21 	 15 	 7
22 	 15 	 10
23 	 5 	 8
24 	 24 	 4
25 	 10 	 14
26 	 14 	 4
27 	 12 	 6
28 	 7 	 5
29 	 8 	 12
30 	 21 	 25
31 	 9 	 9
32 	 25 	 23
33 	 5 	 12
34 	 8 	 5
35 	 4 	 2
36 	 6 	 4
37 	 17 	 4
38 	 4 	 2
39 	 23 	 7
40 	 5 	 3
41 	 6 	 0
42 	 7 	 1
43 	 17 	 4
44 	 2 	 1
45 	 14 	 1
46 	 10 	 1
47 	 15 	 2
48 	 3 	 7
49 	 8 	 1
50 	 6 	 2
51 	 2 	 0
52 	 10 	 1
53 	 2 	 0
54 	 3 	 0
55 	 3 	 0
56 	 4 	 1
57 	 8 	 0
58 	 4 	 0
59 	 4 	 0
60 	 1 	 1
61 	 2 	 0
62 	 17 	 0
63 	 2 	 0
64 	 2 	 0
65 	 3 	 0
66 	 6 	 1
67 	 3 	 0
68 	 7 	 0
69 	 4 	 0
70 	 2 	 0
71 	 8 	 0
72 	 4 	 0
73 	 1 	 1
74 	 3 	 0
75 	 6 	 1
76 	 1 	 0
77 	 1 	 0
78 	 0 	 0
79 	 4 	 0
80 	 5 	 0
81 	 2 	 0
82 	 0 	 0
83 	 1 	 1
8

In [ ]:
def create_table(info, output_file):
    write_text = []
    write_text.append(heading)

    for inf in info:
        line = []
#         print(inf)
        for l_title in titles:
            if inf[0] == l_title:
                line.append(1)
            else: 
                line.append(0)
        
        line.append(inf[2])
        for l_title in titles:
            if inf[1] == l_title:
                line.append(1)
            else: 
                line.append(0)
        
        line.append(inf[3])
        
#         print(line)
        write_text.append(line)
    
    
    with open(output_file, 'w') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(write_text)

In [ ]:
create_table(table_info, irr_table_output_file)

In [ ]:
column_names = []
index_names = []
heading_len = len(heading)
for i in range(0, int(heading_len/2)-1):
#     print(redone_label_initial+' '+heading[i])
    column_names.append(redone_label_initial+' '+heading[i])
#     print(orginal_label_initial+' '+heading[i+int(heading_len/2)])
    index_names.append(orginal_label_initial+' '+heading[i+int(heading_len/2)])

In [ ]:
with open(irr_table_output_file) as csv_file:
    irr_table = pd.read_csv(csv_file)

# print(irr_table)

In [ ]:
row, column = irr_table.shape
half_column = int(column/2)

In [ ]:
irr_matrix = []
for i in range (half_column, column-1):
    matrix_row = []
    for j in range(0, half_column-1):
        cmp_df = irr_table.iloc[:,[i,j]]
        cmp_df.columns = ['one', 'two']
        matrix_row.append(np.sum((cmp_df['one'] == 1) & (cmp_df['two'] == 1)))
    irr_matrix.append(matrix_row)
    

In [ ]:
matrix_df = pd.DataFrame(np.array(irr_matrix), columns=column_names, index=index_names)
export_csv = matrix_df.to_csv(irr_matrix_output_file, header=True)

In [ ]:
matrix_df

In [ ]:
def calculate_cohen_kappa(conf_mat, exclude=None):
    cm = conf_mat.copy()
    if exclude:
        cm = cm.drop(exclude[0], axis=1)
        cm = cm.drop(exclude[1], axis=0)
#     print(cm)
    
    cm = np.array(cm)
    sum_total = np.sum(cm)
    sum_row = np.sum(cm, axis=1)
    sum_column = np.sum(cm, axis=0)
    p_c = np.sum(np.multiply(sum_row, sum_column))/(sum_total**2)
    p_o = np.trace(cm)/sum_total
    k = (p_o - p_c)/(1 - p_c)
    
    return k

In [ ]:
k = calculate_cohen_kappa(matrix_df)
print(k)

In [ ]:
k = calculate_cohen_kappa(matrix_df, exclude=[['Jimmy No label'],['Erica No label']])
print(k)

In [ ]:
k = calculate_cohen_kappa(matrix_df, exclude=[['Jimmy No label','Jimmy Exploring'],['Erica No label','Erica Exploring']])
print(k)

### Test

In [ ]:
tm = [[80,15,0,30,0,2,0,0],
[16,140,0,62,10,21,0,1],
[7,1,20,10,0,0,1,0],
[16,22,5,343,39,79,5,11],
[0,4,0,28,95,10,0,0],
[1,9,0,55,21,355,0,0],
[0,0,1,0,0,0,3,0],
[0,0,0,11,47,80,0,0]]
kk = calculate_cohen_kappa(np.array(tm))
print(kk)

In [ ]:
tm_df = pd.DataFrame(np.array(tm), columns=['A','B','C','D','E','F','G','H'], index=['a','b','c','d','e','f','g','h'])
tm_df
# tm_df.drop(['b'], axis=0)

In [ ]:
calculate_cohen_kappa(tm_df, exclude=[['D','H'],['d','h']])

In [27]:
all_titles = set([])
for subdir, dirs, files in os.walk(redone_label_dir):
    ind = 0
    for filename in files:
        name_extension = get_name_and_extension(filename)
        name = name_extension['name']
        ext = name_extension['ext'].lower()

        if ext == 'json':
            print(ind, ":", name)
            if os.path.isfile(redone_label_dir+filename):
                print('----> ', filename)
                with open(redone_label_dir+filename, 'r') as redone_label_data:
                    redone_labels = json.load(redone_label_data)
                    with open(redone_label_dir+filename, 'r') as original_label_data:
                        original_labels = json.load(original_label_data)
                        
                        for label in redone_labels:
                            if 'title' in label:
                                all_titles.add(label['title'])
                                
                                
print(all_titles)

0 : 2593668385
---->  2593668385.json
0 : 3183541562
---->  3183541562.json
{'Kill', 'Team Fight', 'Push', 'Gank', 'Farm', 'Group Fight', 'Roshan', 'Roam', 'Split Push'}


In [57]:
n = {'a':1, 'b':2}
for x in n:
    print(x)

a
b
